In [3]:
import sys
import json
import nltk

# Load in unstructured data from wherever you've saved it

DATA = '/Users/fabiola/Desktop/PUC/extracao-dados-web-sociais/news/timoreilly.buzz'
data = json.loads(open(DATA).read())

QUERY_TERMS = ['government']

all_posts = [post['content'].lower().split() for post in data]

# Provides tf/idf/tf_idf abstractions

tc = nltk.TextCollection(all_posts)

relevant_posts = []

for idx in range(len(all_posts)):
    score = 0
    for term in [t.lower() for t in QUERY_TERMS]:
        score += tc.tf_idf(term, all_posts[idx])
    if score > 0:
        relevant_posts.append({'score': score, 'title': data[idx]['title'],
                              'link': data[idx]['link']})

# Sort by score and display results

relevant_posts = sorted(relevant_posts, key=lambda p: p['score'], reverse=True)
for post in relevant_posts:
    print(post['title'])
    print('\tLink: %s' % (post['link'], ))
    print('\tScore: %s' % (post['score'], ))

Clay Johnson just sent me this note, and I want to share it more broadly: I'm really disappointed...
	Link: http://www.google.com/buzz/107033731246200681024/4ae7i4Hshng
	Score: 0.07360282010402343
Clay Johnson just sent me this note, and I want to share it more broadly: I'm really disappointed...
	Link: http://www.google.com/buzz/107033731246200681024/4ae7i4Hshng
	Score: 0.07360282010402343
The contrast between CAFE standards (see How Will New CAFE Standards Change the Cars We Drive? ht...
	Link: http://www.google.com/buzz/107033731246200681024/bNxTmnxGbWs
	Score: 0.07062896878668914
Fascinating "government as platform" hallway discussion with John Perry Barlow at Economist Innov...
	Link: http://www.google.com/buzz/107033731246200681024/Wf2fGCecM1K
	Score: 0.03478740253672749
The Importance of Defaults I've been pushing the idea that one of the big lessons government can ...
	Link: http://www.google.com/buzz/107033731246200681024/C1Yh6MLmDeS
	Score: 0.019977908313949216
Newt Gingrich 

In [5]:
import sys
import json
import nltk

# Load in textual data from wherever you've saved it

#DATA = sys.argv[1]
#data = json.loads(open(DATA).read())

all_posts = [post['content'].lower().split() for post in data]

# Provides tf/idf/tf_idf abstractions for scoring

tc = nltk.TextCollection(all_posts)

# Compute a term-document matrix such that td_matrix[doc_title][term]
# returns a tf-idf score for the term in the document

td_matrix = {}
for idx in range(len(all_posts)):
    post = all_posts[idx]
    fdist = nltk.FreqDist(post)

    doc_title = data[idx]['title']
    link = data[idx]['link']
    td_matrix[(doc_title, link)] = {}

    for term in fdist:
        td_matrix[(doc_title, link)][term] = tc.tf_idf(term, post)

# Build vectors such that term scores are in the same positions...

distances = {}
for (title1, link1) in td_matrix.keys():

    distances[(title1, link1)] = {}
    (max_score, most_similar) = (0.0, (None, None))

    for (title2, link2) in td_matrix.keys():

        # Take care not to mutate the original data structures
        # since we're in a loop and need the originals multiple times

        terms1 = td_matrix[(title1, link1)].copy()
        terms2 = td_matrix[(title2, link2)].copy()

        # Fill in "gaps" in each map so vectors of the same length can be computed

        for term1 in terms1:
            if term1 not in terms2:
                terms2[term1] = 0

        for term2 in terms2:
            if term2 not in terms1:
                terms1[term2] = 0

        # Create vectors from term maps

        v1 = [score for (term, score) in sorted(terms1.items())]
        v2 = [score for (term, score) in sorted(terms2.items())]

        # Compute similarity amongst documents

        distances[(title1, link1)][(title2, link2)] = \
            nltk.cluster.util.cosine_distance(v1, v2)

        if link1 == link2:
            continue
        if distances[(title1, link1)][(title2, link2)] > max_score:
            (max_score, most_similar) = (distances[(title1, link1)][(title2,
                                         link2)], (title2, link2))

    print( '''Most similar to %s (%s)
\t%s (%s)
\tscore %s
''' % (title1, link1,
            most_similar[0], most_similar[1], max_score))

Most similar to IMG_20101126_130437 (http://www.google.com/buzz/107033731246200681024/JFrxdGvTJQZ)
	Our Quarrels with the World I just came across a fabulous quote from Randall Jarrell in an email ... (http://www.google.com/buzz/107033731246200681024/anjcWYmhFdh)
	score 1.0

Most similar to Our Quarrels with the World I just came across a fabulous quote from Randall Jarrell in an email ... (http://www.google.com/buzz/107033731246200681024/anjcWYmhFdh)
	IMG_20101126_130437 (http://www.google.com/buzz/107033731246200681024/JFrxdGvTJQZ)
	score 1.0

Most similar to the world has moved on (http://www.google.com/buzz/107033731246200681024/DGkWH34N6Q9)
	IMG_20101126_130437 (http://www.google.com/buzz/107033731246200681024/JFrxdGvTJQZ)
	score 1.0

Most similar to eucalyptus in the rain (http://www.google.com/buzz/107033731246200681024/4A6M92LJoeg)
	IMG_20101126_130437 (http://www.google.com/buzz/107033731246200681024/JFrxdGvTJQZ)
	score 1.0

Most similar to Fabulous piece via @sarawinge in bac

Most similar to At the Economist #IdeasEconomy meeting, in a session entitled The World Is Not Flat? Ken Cukier p... (http://www.google.com/buzz/107033731246200681024/bz7vKY6FWhJ)
	IMG_20101126_130437 (http://www.google.com/buzz/107033731246200681024/JFrxdGvTJQZ)
	score 1.0

Most similar to The Importance of Defaults I've been pushing the idea that one of the big lessons government can ... (http://www.google.com/buzz/107033731246200681024/C1Yh6MLmDeS)
	IMG_20101126_130437 (http://www.google.com/buzz/107033731246200681024/JFrxdGvTJQZ)
	score 1.0

Most similar to Newt Gingrich apparentlycompared the effect of Dems passing #HCR to Johnson passing civil rights ... (http://www.google.com/buzz/107033731246200681024/CvTw6R6bk5Z)
	IMG_20101126_130437 (http://www.google.com/buzz/107033731246200681024/JFrxdGvTJQZ)
	score 1.0

Most similar to The DIY/hacker impulse gets on the radar of the Economist: In this blog, our correspondents tinke... (http://www.google.com/buzz/107033731246200681024/ib1B9

In [12]:
import os
import sys
import shutil
import webbrowser
import json
from operator import itemgetter
import nltk
import numpy

# Load in textual data from wherever you've saved it

#DATA = sys.argv[1]
#data = json.loads(open(DATA).read())

# HTML templmates that we'll inject Protovis consumable data into
HTML_TEMPLATES = ['matrix_diagram.html', 
                  'arc_diagram.html']

all_posts = [post['content'].lower().split() for post in data]

# Provides tf/idf/tf_idf abstractions for scoring

tc = nltk.TextCollection(all_posts)

# Compute a term-document matrix such that td_matrix[doc_title][term]
# returns a tf-idf score for the term in the document

td_matrix = {}
for idx in range(len(all_posts)):
    post = all_posts[idx]
    fdist = nltk.FreqDist(post)

    doc_title = data[idx]['title']
    link = data[idx]['link']
    td_matrix[(doc_title, link)] = {}

    for term in fdist:
        td_matrix[(doc_title, link)][term] = tc.tf_idf(term, post)

# Build vectors such that term scores are in the same positions...

distances = {}
for (title1, link1) in td_matrix.keys():

    distances[(title1, link1)] = {}

    for (title2, link2) in td_matrix.keys():

        if link1 == link2:
            continue

        # Take care not to mutate the original data structures
        # since we're in a loop and need the originals multiple times

        terms1 = td_matrix[(title1, link1)].copy()
        terms2 = td_matrix[(title2, link2)].copy()

        # Fill in "gaps" in each map so vectors of the same length can be computed

        for term1 in terms1:
            if term1 not in terms2:
                terms2[term1] = 0

        for term2 in terms2:
            if term2 not in terms1:
                terms1[term2] = 0

        # Create vectors from term maps

        v1 = [score for (term, score) in sorted(terms1.items())]
        v2 = [score for (term, score) in sorted(terms2.items())]

        # Compute similarity amongst documents

        distances[(title1, link1)][(title2, link2)] = \
            nltk.cluster.util.cosine_distance(v1, v2)

# Compute the standard deviation for the distances as a basis of automated thresholding

std = numpy.std([distances[k1][k2] for k1 in distances for k2 in distances[k1]])

similar = []
keys = td_matrix.keys()

for k1 in keys:
    for k2 in keys:
        if k1 == k2:
            continue

        d = distances[k1][k2]
        print(d, (std/2))
        #if d < std / 2 and d > 0.000001:  # call them similar
        if d > 0.9999:  # call them similar
            (title1, link1) = k1
            (title2, link2) = k2
            similar.append((k1, k2, distances[k1][k2]))

# Emit output expected by Protovis.

nodes = {}
node_idx = 0
edges = []
for s in similar:
    if s[0] not in nodes:
        nodes[s[0]] = node_idx
        node_idx += 1

    node0 = nodes[s[0]]

    if s[1] not in nodes:
        nodes[s[1]] = node_idx
        node_idx += 1

    node1 = nodes[s[1]]

    edges.append({'source': node0, 'target': node1, 'value': s[2] * 1000})

nodes = [{'nodeName': title, 'nodeUrl': url} for ((title, url), idx) in
         sorted(nodes.items(), key=itemgetter(1))]

json_data = {'nodes': nodes, 'links': edges}
print(json_data)
# This json_data is consumed by matrix_diagram.html
if not os.path.isdir('out'):
    os.mkdir('out')

# HTML_TEMPLATE references some Protovis scripts, which we can
# simply copy into out/

shutil.rmtree('out/protovis-3.2', ignore_errors=True)

shutil.copytree('protovis-3.2',
                'out/protovis-3.2')


for template in HTML_TEMPLATES:
    html = open(template).read() % (json.dumps(json_data),)
    f = open(os.path.join(os.getcwd(), 'out', os.path.basename(template)), 'w')
    f.write(html)
    f.close()

    print('Data file written to: %s' % f.name)
    webbrowser.open('file://' + f.name)

1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754


1.0 0.013644133092906754
1.0 0.013644133092906754
0.948783248460337 0.013644133092906754
0.9909869930427017 0.013644133092906754
0.9752178028826929 0.013644133092906754
0.9774789159706693 0.013644133092906754
1.0 0.013644133092906754
0.9515995300590822 0.013644133092906754
1.0 0.013644133092906754
0.9853795160180617 0.013644133092906754
1.0 0.013644133092906754
0.9697924465348475 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
0.9752809224510314 0.013644133092906754
1.0 0.013644133092906754
0.9384987726291644 0.013644133092906754
0.9697391397320331 0.013644133092906754
0.9760070219263699 0.013644133092906754
0.9557351603192094 0.013644133092906754
0.981273097644257 0.013644133092906754
0.9950386320028869 0.013644133092906754
0.9779987904581491 0.013644133092906754
0.9738482657520495 0.013644133092906754
0.973565748185575 0.013644133092906754
0.9904410489167909 0.013644133092906754
1.0 0.013644133092906754
1.0 0.01

1.0 0.013644133092906754
0.992455569294647 0.013644133092906754
0.968957822039873 0.013644133092906754
1.0 0.013644133092906754
0.9959993048167902 0.013644133092906754
0.9914874901075492 0.013644133092906754
0.9996311780372884 0.013644133092906754
0.9975397146968603 0.013644133092906754
0.9992936144358031 0.013644133092906754
0.9995688180001865 0.013644133092906754
0.9929312675527874 0.013644133092906754
0.9957530671376609 0.013644133092906754
0.9995637559985266 0.013644133092906754
0.9977486278841727 0.013644133092906754
0.997212091590757 0.013644133092906754
0.9974640967596898 0.013644133092906754
0.998043347770881 0.013644133092906754
0.9960499757503662 0.013644133092906754
0.9991022299485016 0.013644133092906754
0.9985942628027095 0.013644133092906754
0.9988008153969478 0.013644133092906754
0.9929159831787866 0.013644133092906754
0.9958026239415741 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.01364413

0.912236583342578 0.013644133092906754
0.9574766833944274 0.013644133092906754
0.9542858376229505 0.013644133092906754
0.9570426993248449 0.013644133092906754
0.9935642939394234 0.013644133092906754
0.9396106976506245 0.013644133092906754
0.9476496519649777 0.013644133092906754
0.9699044146588552 0.013644133092906754
0.9756909592550458 0.013644133092906754
0.9375814385494062 0.013644133092906754
0.9485871101145228 0.013644133092906754
0.9492827635334022 0.013644133092906754
0.9108799007527947 0.013644133092906754
0.9740748376559553 0.013644133092906754
0.9577102757290376 0.013644133092906754
0.9759492631703897 0.013644133092906754
0.9424143618050014 0.013644133092906754
0.9679325277818724 0.013644133092906754
1.0 0.013644133092906754
0.9510372955087879 0.013644133092906754
0.9901072443330339 0.013644133092906754
0.9939619910846377 0.013644133092906754
0.944735008796302 0.013644133092906754
0.9573325030618721 0.013644133092906754
0.9484550905589766 0.013644133092906754
0.941287380536760

0.973565748185575 0.013644133092906754
0.918110862579203 0.013644133092906754
1.0 0.013644133092906754
0.9252615201791282 0.013644133092906754
0.9933125757383048 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
0.9307891349245773 0.013644133092906754
1.0 0.013644133092906754
0.8804998383826261 0.013644133092906754
0.9731671229153764 0.013644133092906754
0.9384270372110005 0.013644133092906754
0.9174207018528463 0.013644133092906754
0.9361142370335414 0.013644133092906754
0.9458894563779803 0.013644133092906754
0.9595115755078312 0.013644133092906754
0.9243650302871299 0.013644133092906754
0.9613883528798647 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
0.9710547833577816 0.013644133092906754
0.92139492260689 0.013644133092906754
1.0 0.013644133092906754
0.9252472599267897 0.013644133092906754
0.9049219109276899 0.013644133092906754
0.9574074435426647 0.013644133092906754
0.947935751704962 0

0.9767643241382693 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
0.9627521483902439 0.013644133092906754
1.0 0.013644133092906754
0.8890351615196486 0.013644133092906754
0.9657855569662229 0.013644133092906754
0.9371736467114791 0.013644133092906754
0.912236583342578 0.013644133092906754
0.9249086173796512 0.013644133092906754
0.9618901367764033 0.013644133092906754
0.9628809343978537 0.013644133092906754
0.9393057402885312 0.013644133092906754
0.9049219109276899 0.013644133092906754
0.9546185021395692 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
0.9777255638326399 0.013644133092906754
0.8919984499140696 0.013644133092906754
1.0 0.013644133092906754
0.9367758773134969 0.013644133092906754
0.9320946074459342 0.013644133092906754
0.9488482913478938 0.013644133092906754
0.9396024775082888 0.013644133092906754
0.9630700760679595 0.013644133092906754
0.9384642768149771 0.013644133092906754
0.9284946950790709 0.01364

0.9827517449425283 0.013644133092906754
0.9695714541499717 0.013644133092906754
0.9852439047811274 0.013644133092906754
0.9334144362144638 0.013644133092906754
0.9921294365571953 0.013644133092906754
0.9797691847580103 0.013644133092906754
0.9954162528200209 0.013644133092906754
0.9929843001156268 0.013644133092906754
0.9940705893449066 0.013644133092906754
1.0 0.013644133092906754
0.9355601446020165 0.013644133092906754
0.9897102144722554 0.013644133092906754
0.9962332724817143 0.013644133092906754
0.9587959708198791 0.013644133092906754
0.9678436922841931 0.013644133092906754
0.9489779539350837 0.013644133092906754
0.9441519577606512 0.013644133092906754
1.0 0.013644133092906754
0.93894986881215 0.013644133092906754
1.0 0.013644133092906754
0.9827650257191348 0.013644133092906754
0.960185112267645 0.013644133092906754
1.0 0.013644133092906754
0.954147173260964 0.013644133092906754
0.9929312675527874 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.01364413

1.0 0.013644133092906754
0.8888221085853296 0.013644133092906754
0.9738313091472521 0.013644133092906754
0.9062676231654959 0.013644133092906754
0.9375814385494062 0.013644133092906754
0.9432578862986841 0.013644133092906754
0.9618019343962658 0.013644133092906754
0.9753896129374817 0.013644133092906754
0.9487190998646435 0.013644133092906754
0.910325192953227 0.013644133092906754
0.9557726311983551 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
0.9830731431348761 0.013644133092906754
0.9157352133144846 0.013644133092906754
1.0 0.013644133092906754
0.9270221071697744 0.013644133092906754
0.9193241497845412 0.013644133092906754
0.9626842404285592 0.013644133092906754
0.9754908845066895 0.013644133092906754
0.9475504003775986 0.013644133092906754
0.9827517449425283 0.013644133092906754
0.9492889724716715 0.013644133092906754
0.946508572861091 0.013644133092906754
0.9660753884421694 0.013644133092906754
0.9685106838021097 0.0136441330929067

0.9610707066168674 0.013644133092906754
0.9839946584651963 0.013644133092906754
0.9963608153274445 0.013644133092906754
0.9707787511608595 0.013644133092906754
0.9871582985742301 0.013644133092906754
1.0 0.013644133092906754
0.9571098437987849 0.013644133092906754
0.9829310487133426 0.013644133092906754
0.9963131719629981 0.013644133092906754
0.9630433569298428 0.013644133092906754
0.9057240237523068 0.013644133092906754
0.924433783490123 0.013644133092906754
0.9299018581139925 0.013644133092906754
1.0 0.013644133092906754
0.963706302664867 0.013644133092906754
1.0 0.013644133092906754
0.9827919923756645 0.013644133092906754
0.9778600838150354 0.013644133092906754
1.0 0.013644133092906754
0.9666930835854644 0.013644133092906754
0.9985942628027095 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
1.0 0.013644133092906754
0.9640381290960759 0.013644133092906754
1.0 0.013644133092906754
0.9534476530154834 0.013644133092906754
0.9357793032083451 0.013644133092906754
0.

Data file written to: /Users/fabiola/Desktop/PUC/extracao-dados-web-sociais/news/out/arc_diagram.html
